In [1]:
import numpy as np
import scipy.optimize as opt
import seaborn as sns
import emcee
from multiprocessing import Pool

In [2]:
def read_signal(file):
    ret_ls = []
    with open(file, 'rb') as f:
        while True:
            try: ret_ls.append(np.load(f))
            except EOFError:
                break
    return ret_ls

In [3]:
signal_path = './signal_z3/'
rp_auto, signal_auto, rp_cross, signal_cross = read_signal(f'{signal_path}/signal.npy')
cov = np.load(f'{signal_path}/cov.npy')

In [4]:
rp_auto

array([ 0.32556026,  0.41129803,  0.51961524,  0.65645828,  0.82933956,
        1.04774991,  1.32367962,  1.67227667,  2.11267834,  2.6690618 ,
        3.37197138,  4.25999539,  5.38188457,  6.79922837,  8.58983612,
       10.85200857, 13.70993443, 17.32050808, 21.88194273, 27.64465197,
       34.92499691, 44.12265385, 55.74255561, 70.4226114 , 88.96872671])

In [58]:
auto_range = (10, 20)
cross_range = (0, 10)

In [59]:
l_a, r_a = auto_range[0], auto_range[1]
l_c, r_c = cross_range[0], cross_range[1]

auto_size = len(rp_auto)
cross_size = len(rp_cross)
rp_auto = rp_auto[l_a:r_a]
rp_cross = rp_cross[l_c:r_c]
signal_auto = signal_auto[l_a:r_a]
signal_cross = signal_cross[l_c:r_c]

# cov = np.vstack((
#     np.hstack((cov[l_c:r_c, l_c:r_c], cov[l_c:r_c, cross_size+l_a:cross_size+r_a])),
#     np.hstack((cov[cross_size+l_a:cross_size+r_a, l_c:r_c], cov[cross_size+l_a:cross_size+r_a, cross_size+l_a:cross_size+r_a]))
# ))

In [60]:
Cmm = cov[l_c:r_c, l_c:r_c]
Cqm = cov[l_c:r_c, cross_size+l_a:cross_size+r_a]
Cmq = cov[cross_size+l_a:cross_size+r_a, l_c:r_c]
Cqq = cov[cross_size+l_a:cross_size+r_a, cross_size+l_a:cross_size+r_a]

## maximize the ln likely hood

- define $w_{qq} - b w_{qm}$ as $\delta$, $C =  Cov(\delta, \delta)$
- target: $\chi^2 = \delta^T C^{-1} \delta$, minimize $-\ln |C| - \chi^2$.
- note that $C$ also depends on $b$.

In [61]:
E = Cmm
F = -(Cqm + Cmq)
G = Cqq

In [ ]:
def ln_like(b):
    C = E*b**2 + F*b + G
    delta = signal_auto - signal_cross*b
    chi2 = delta.reshape((1, -1))@np.linalg.inv(C)@delta.reshape((-1, 1))
    return -chi2.flatten()[0] + -np.log(np.linalg.det(C))       # log L propto -chi^2 - log |C|

In [63]:
opt.minimize_scalar(lambda b: -ln_like(b), bounds = (0, 3))

 message: Solution found.
 success: True
  status: 0
     fun: 39.9663524244838
       x: 2.6891569395551214
     nit: 12
    nfev: 12

In [64]:
pool = Pool(30)
init_state = np.random.uniform(1, 3, 40)
sampler = emcee.EnsembleSampler(40, 1, ln_like, pool)
sampler.run_mcmc(init_state.reshape(-1, 1), 3000)
sampler.get_autocorr_time()

array([24.7756683])

In [65]:
chain = sampler.get_chain(discard=500, flat = True)
res = np.percentile(chain, [50-34.1, 50, 50+34.1])
print(res[1], res[1]-res[0], res[2]-res[1])

2.7235811486422534 0.21106294234247303 0.24236114814056808
